In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 4.8 MB/s eta 0:00:00ta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=fe27975270d002401bdab9dbc8bf43619620196ad339bc2287b0141d5fdf68bb
  Stored in directory: /home/ubuntu/.cache/pip/wheels/95/13/41/f7f135ee114175605fb4f0a89e7389f3742aa6c1e1a5bcb657
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, FloatType, LongType, StringType, DoubleType
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.functions import col
from pyspark.ml import Pipeline, Transformer
from pyspark.ml.feature import StringIndexer, VectorAssembler, Imputer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
import pyspark.sql.functions as F
from itertools import combinations
import os

## Check Python Path

In [4]:
import sys
sys.executable

'/home/ubuntu/anaconda3/bin/python'

In [5]:
DATA_FOLDER = "data"

NUMBER_OF_FOLDS = 3
SPLIT_SEED = 7576
TRAIN_TEST_SPLIT = 0.8

## Function for data reading

In [6]:

def read_data(spark: SparkSession) -> DataFrame:
    """
    read data; since the data has the header we let spark guess the schema
    """
    
    # Read the Titanic CSV data into a DataFrame
    titanic_data = spark.read \
        .format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .load(os.path.join(DATA_FOLDER,"heart_disease.csv"))

    return titanic_data

## Writing new Transformer type class : adding cross product of features

In [7]:
class PairwiseProduct(Transformer):

    def __init__(self, inputCols, outputCols):
        self.__inputCols = inputCols
        self.__outputCols = outputCols

        self._paramMap = self._params = {}

    def _transform(self, df):
        for cols, out_col in zip(self.__inputCols, self.__outputCols):
            df = df.withColumn(out_col, col(cols[0]) * col(cols[1]))
        return df

## The ML pipeline

In [8]:


def pipeline(data: DataFrame):

    """
    every attribute that is numeric is non-categorical; this is questionable
    """

    numeric_features = [f.name for f in data.schema.fields if isinstance(f.dataType, (DoubleType, FloatType, IntegerType, LongType))]
    string_features = [f.name for f in data.schema.fields if isinstance(f.dataType, StringType)]
    #numeric_features.remove("PassengerId")
    #numeric_features.remove("Survived")
    #string_features.remove("Name")
    print(numeric_features)
    print(string_features)

    # Fill missing values for string columns with a placeholder before indexing
    data = data.fillna({col: 'null' for col in string_features})
    
    # Index string features
    indexed_string_columns = [f"{v}Index" for v in string_features]
    indexers = [StringIndexer(inputCol=col, outputCol=indexed_col, handleInvalid='keep') for col, indexed_col in zip(string_features, indexed_string_columns)]

    # Impute missing values for indexed string columns
    imputed_columns_string = [f"Imputed{v}" for v in indexed_string_columns]
    imputer_string = Imputer(inputCols=indexed_string_columns, outputCols=imputed_columns_string, strategy="mode")

    
    # numeric columns
    imputed_columns_numeric = [f"Imputed{v}" for v in numeric_features]
    imputer_numeric = Imputer(inputCols=numeric_features, outputCols=imputed_columns_numeric, strategy = "mean")

    # Create all pairwise products of numeric features
    #all_pairs = [v for v in combinations(imputed_columns_numeric, 2)]
    #pairwise_columns = [f"{col1}_{col2}" for col1, col2 in all_pairs]
    #pairwise_product = PairwiseProduct(inputCols=all_pairs, outputCols=pairwise_columns)

    # Assemble feature columns into a single feature vector
    assembler = VectorAssembler(
        inputCols=imputed_columns_numeric + imputed_columns_string, 
        outputCol="features"
        )

    # Create a list of pipeline stages
    stages = indexers + [imputer_string, imputer_numeric, assembler]
    
    # Create and fit the pipeline
    pipeline = Pipeline(stages=stages)
    model = pipeline.fit(data)
    
    # Transform the data
    transformed_data = model.transform(data)
    
    return transformed_data

    
    

    

In [9]:
def main():
    # Create a Spark session
    spark = SparkSession.builder \
        .appName("Predict Titanic Survival") \
        .getOrCreate()

    try:
        # Read data
        data = read_data(spark)
        
        # Print schema and preview the data
        data.printSchema()
        data.show(5)

        # Apply the pipeline
        transformed_data = pipeline(data)
        
        # Show the transformed data, including the imputed columns
        columns_to_show = [col for col in transformed_data.columns if col.startswith("Imputed")]
        transformed_data.select(columns_to_show).show(truncate=False)
        
    finally:
        # Stop the Spark session
        spark.stop()

In [10]:

main()

JAVA_HOME is not set


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.